# Gather Data and Exploratory Data Analysis (EDA)

## Gather Data

In [1]:
# import libraries
from IPython.display import display_html
from scipy.sparse import SparseEfficiencyWarning
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.nonparametric.smoothers_lowess import lowess
from statsmodels.tsa.filters.hp_filter import hpfilter
import dataframe_image as dfi
import math
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import requests
import scipy
import seaborn as sns
import warnings

# FRED API setting
API_KEY = 'MY_FRED_API_KEY'
BASE = 'https://api.stlouisfed.org/fred/series/observations'

# Data Period
START = '1800-01-01'
END = '2099-12-31'

# a function to fetch series via FRED API
def fetch_series(series_id, start=START, api_key=API_KEY):
    params = {
        'series_id': series_id,
        'api_key': api_key,
        'file_type': 'json',
        'observation_start': start,
    }
    r = requests.get(BASE, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    obs = js.get('observations', [])
    if not obs:
        return pd.Series(name=series_id, dtype='float64')
    s = pd.Series(
        [None if o['value'] in ('.','') else float(o['value']) for o in obs],
        index=pd.to_datetime([o['date'] for o in obs]),
        name=series_id
    )
    # Remove duplicates in the date index (keep the last one) and sort
    s = s[~s.index.duplicated(keep='last')].sort_index()
    return s

# a function to display multiple dataframes side by side
def display_side_by_side(*args, titles=()):
    html_str = ''
    for i, df in enumerate(args):
        title = titles[i] if i < len(titles) else f'DF{i+1}'
        html_str += f'<div style="display:inline-block; margin-right:20px;">'
        html_str += f'<h3>{title}</h3>'
        html_str += df.to_html()
        html_str += '</div>'
    display_html(html_str, raw=True)

### Inflation
- Core PCE

In [2]:
df_PCEPILFE_before = pd.DataFrame(fetch_series('PCEPILFE'))

# Copy the dataframe
df_PCEPILFE_after = df_PCEPILFE_before.copy()

# Create YoY series (% change from a year ago)
df_PCEPILFE_after['PCEPILFE_YoY'] = df_PCEPILFE_after['PCEPILFE'].pct_change(periods=12) * 100

# Create MoM annualized series (% change from a month ago)
df_PCEPILFE_after['PCEPILFE_MoM'] = df_PCEPILFE_after['PCEPILFE'].pct_change() * 100 * 12

# Drop rows where any row has NaN
df_PCEPILFE_after = df_PCEPILFE_after.dropna()

# Extract the spcific columns
df_PCEPILFE_after = df_PCEPILFE_after[['PCEPILFE_YoY', 'PCEPILFE_MoM']].copy()

print(df_PCEPILFE_after.isnull().sum())

display_side_by_side(
    df_PCEPILFE_before.head(), df_PCEPILFE_after.head(), df_PCEPILFE_before.tail(), df_PCEPILFE_after.tail(), 
    titles=['Raw Data (Head)', 'Cleaned Data (Head)', 'Raw Data (Tail)', 'Cleaned Data (Tail)']
)

PCEPILFE_YoY    0
PCEPILFE_MoM    0
dtype: int64


Raw Data (Head) 
 
 
 
 PCEPILFE 
 
 
 
 
 1959-01-01 
 15.501 
 
 
 1959-02-01 
 15.513 
 
 
 1959-03-01 
 15.531 
 
 
 1959-04-01 
 15.570 
 
 
 1959-05-01 
 15.589 
 
 
 Cleaned Data (Head) 
 
 
 
 PCEPILFE_YoY 
 PCEPILFE_MoM 
 
 
 
 
 1960-01-01 
 2.070834 
 0.303452 
 
 
 1960-02-01 
 2.185264 
 2.275313 
 
 
 1960-03-01 
 2.073273 
 0.075700 
 
 
 1960-04-01 
 2.003854 
 2.195168 
 
 
 1960-05-01 
 2.046315 
 1.964488 
 
 
 Raw Data (Tail) 
 
 
 
 PCEPILFE 
 
 
 
 
 2025-03-01 
 125.118 
 
 
 2025-04-01 
 125.329 
 
 
 2025-05-01 
 125.579 
 
 
 2025-06-01 
 125.909 
 
 
 2025-07-01 
 126.253 
 
 
 Cleaned Data (Tail) 
 
 
 
 PCEPILFE_YoY 
 PCEPILFE_MoM 
 
 
 
 
 2025-03-01 
 2.699686 
 1.142409 
 
 
 2025-04-01 
 2.610938 
 2.023690 
 
 
 2025-05-01 
 2.732352 
 2.393700 
 
 
 2025-06-01 
 2.774467 
 3.153393 
 
 
 2025-07-01 
 2.877235 
 3.278558

### Unemployment
- Unemployment Rate

In [3]:
df_UNRATE_before = pd.DataFrame(fetch_series('UNRATE'))

# Copy the dataframe
df_UNRATE_after = df_UNRATE_before.copy()

print(df_UNRATE_after.isnull().sum())

display_side_by_side(
    df_UNRATE_before.head(), df_UNRATE_after.head(), df_UNRATE_before.tail(), df_UNRATE_after.tail(), 
    titles=['Raw Data (Head)', 'Cleaned Data (Head)', 'Raw Data (Tail)', 'Cleaned Data (Tail)']
)

UNRATE    0
dtype: int64


,UNRATE
1948-01-01,3.4
1948-02-01,3.8
1948-03-01,4.0
1948-04-01,3.9
1948-05-01,3.5
,UNRATE
1948-01-01,3.4
1948-02-01,3.8
1948-03-01,4.0
1948-04-01,3.9


- Noncyclical Rate of Unemployment

In [4]:
df_NROU_before = pd.DataFrame(fetch_series('NROU'))

# Copy the dataframe
df_NROU_after = df_NROU_before.copy()

# Convert a quarterly series into a monthly series using spline interpolation
df_NROU_after = df_NROU_after.resample('MS').interpolate(method='spline', order=3)

print(df_NROU_after.isnull().sum())

display_side_by_side(
    df_NROU_before.head(), df_NROU_after.head(), df_NROU_before.tail(), df_NROU_after.tail(), 
    titles=['Raw Data (Head)', 'Cleaned Data (Head)', 'Raw Data (Tail)', 'Cleaned Data (Tail)']
)

NROU    0
dtype: int64


,NROU
1949-01-01,5.255053
1949-04-01,5.261516
1949-07-01,5.268013
1949-10-01,5.274564
1950-01-01,5.281182
,NROU
1949-01-01,5.255053
1949-02-01,4.909390
1949-03-01,4.916764
1949-04-01,5.261516


### Business Cycles and Policy Regimes
- Recession Indicators

In [5]:
df_USREC_before = pd.DataFrame(fetch_series('USREC'))

# Copy the dataframe
df_USREC_after = df_USREC_before.copy()

print(df_USREC_after.isnull().sum())

display_side_by_side(
    df_USREC_before.head(), df_USREC_after.head(), df_USREC_before.tail(), df_USREC_after.tail(), 
    titles=['Raw Data (Head)', 'Cleaned Data (Head)', 'Raw Data (Tail)', 'Cleaned Data (Tail)']
)

USREC    0
dtype: int64


,USREC
1854-12-01,1.0
1855-01-01,0.0
1855-02-01,0.0
1855-03-01,0.0
1855-04-01,0.0
,USREC
1854-12-01,1.0
1855-01-01,0.0
1855-02-01,0.0
1855-03-01,0.0


- Zero Lower Bound dummy

In [6]:
df_ZLB_before = pd.DataFrame(fetch_series('FEDFUNDS'))

# Copy the dataframe
df_ZLB_after = df_ZLB_before.copy()

# Create dummy series: 1 if FF rate <= 0.25, else 0
df_ZLB_after['ZLB_dummy'] = (df_ZLB_after['FEDFUNDS'] <= 0.25).astype(int)

# Extract the specific column
df_ZLB_after = df_ZLB_after[['ZLB_dummy']]

print(df_ZLB_after.isnull().sum())

display_side_by_side(
    df_ZLB_before.head(), df_ZLB_after.head(), df_ZLB_before.tail(), df_ZLB_after.tail(), 
    titles=['Raw Data (Head)', 'Cleaned Data (Head)', 'Raw Data (Tail)', 'Cleaned Data (Tail)']
)

ZLB_dummy    0
dtype: int64


,FEDFUNDS
1954-07-01,0.80
1954-08-01,1.22
1954-09-01,1.07
1954-10-01,0.85
1954-11-01,0.83
,ZLB_dummy
1954-07-01,0
1954-08-01,0
1954-09-01,0
1954-10-01,0


- COVID-19 Period dummy

In [7]:
# Create a monthly date index from START to END
date_index = pd.date_range(start=START, end=END, freq='MS')

# Initialize the DataFrame with zeros
df_COVID = pd.DataFrame(index=date_index)
df_COVID['COVID_dummy'] = 0

# Define the COVID period (Apr 2020 - Mar 2021)
mask = (df_COVID.index >= '2020-04-01') & (df_COVID.index <= '2021-03-31')

# Assign 1 for the COVID period
df_COVID.loc[mask, 'COVID_dummy'] = 1

print(df_COVID.isnull().sum())

display_side_by_side(
    df_COVID.head(12), df_COVID.loc['2020-01-01':'2020-12-31'], df_COVID.loc['2021-01-01':'2021-12-31'], df_COVID.tail(12), 
    titles=['COVID Dummy (Head)', 'COVID Dummy (2020)', 'COVID Dummy (2021)', 'COVID Dummy (Tail)']
)

COVID_dummy    0
dtype: int64


,COVID_dummy
1800-01-01,0
1800-02-01,0
1800-03-01,0
1800-04-01,0
1800-05-01,0
1800-06-01,0
1800-07-01,0
1800-08-01,0
1800-09-01,0
1800-10-01,0


### Demand
- Real GDP

In [8]:
df_GDPC1_before = pd.DataFrame(fetch_series('GDPC1'))

# Copy the dataframe
df_GDPC1_after = df_GDPC1_before.copy()

# Compute QoQ annualized growth
df_GDPC1_after['GDPC1'] = df_GDPC1_after['GDPC1'].pct_change() * 100 * 4

# Convert a quarterly series into a monthly series using spline interpolation
df_GDPC1_after = df_GDPC1_after.resample('MS').interpolate(method='spline', order=3)

# Drop rows where any row has NaN
df_GDPC1_after = df_GDPC1_after.dropna()

print(df_GDPC1_after.isnull().sum())

display_side_by_side(
    df_GDPC1_before.head(), df_GDPC1_after.head(), df_GDPC1_before.tail(), df_GDPC1_after.tail(), 
    titles=['Raw Data (Head)', 'Cleaned Data (Head)', 'Raw Data (Tail)', 'Cleaned Data (Tail)']
)

GDPC1    0
dtype: int64


,GDPC1
1947-01-01,2182.681
1947-04-01,2176.892
1947-07-01,2172.432
1947-10-01,2206.452
1948-01-01,2239.682
,GDPC1
1947-04-01,-1.060897
1947-05-01,-2.722486
1947-06-01,-2.394977
1947-07-01,-0.819517


- Real Potential GDP

In [9]:
df_GDPPOT_before = pd.DataFrame(fetch_series('GDPPOT'))

# Copy the dataframe
df_GDPPOT_after = df_GDPPOT_before.copy()

# Compute QoQ annualized growth
df_GDPPOT_after['GDPPOT'] = df_GDPPOT_after['GDPPOT'].pct_change() * 100 * 4

# Convert a quarterly series into a monthly series using spline interpolation
df_GDPPOT_after = df_GDPPOT_after.resample('MS').interpolate(method='spline', order=3)

# Drop rows where any row has NaN
df_GDPPOT_after = df_GDPPOT_after.dropna()

print(df_GDPPOT_after.isnull().sum())

display_side_by_side(
    df_GDPPOT_before.head(), df_GDPPOT_after.head(), df_GDPPOT_before.tail(), df_GDPPOT_after.tail(), 
    titles=['Raw Data (Head)', 'Cleaned Data (Head)', 'Raw Data (Tail)', 'Cleaned Data (Tail)']
)

GDPPOT    0
dtype: int64


,GDPPOT
1949-01-01,2254.866716
1949-04-01,2287.486427
1949-07-01,2321.150343
1949-10-01,2352.817952
1950-01-01,2380.500009
,GDPPOT
1949-04-01,5.786544
1949-05-01,4.422671
1949-06-01,4.419234
1949-07-01,5.886621


- Industrial Production

In [10]:
df_INDPRO_before = pd.DataFrame(fetch_series('INDPRO'))

# Copy the dataframe
df_INDPRO_after = df_INDPRO_before.copy()

# Compute monthly annualized growth
df_INDPRO_after['INDPRO'] = df_INDPRO_after['INDPRO'].pct_change() * 100 * 12

# Drop rows where any row has NaN
df_INDPRO_after = df_INDPRO_after.dropna()

print(df_INDPRO_after.isnull().sum())

display_side_by_side(
    df_INDPRO_before.head(), df_INDPRO_after.head(), df_INDPRO_before.tail(), df_INDPRO_after.tail(), 
    titles=['Raw Data (Head)', 'Cleaned Data (Head)', 'Raw Data (Tail)', 'Cleaned Data (Tail)']
)

INDPRO    0
dtype: int64


,INDPRO
1919-01-01,4.8654
1919-02-01,4.6504
1919-03-01,4.5160
1919-04-01,4.5966
1919-05-01,4.6235
,INDPRO
1919-02-01,-53.027500
1919-03-01,-34.680888
1919-04-01,21.417183
1919-05-01,7.022582


- Retail Sales

In [11]:
df_RSAFS_before = pd.DataFrame(fetch_series('RSAFS'))

# Copy the dataframe
df_RSAFS_after = df_RSAFS_before.copy()

# Compute monthly annualized growth
df_RSAFS_after['RSAFS'] = df_RSAFS_after['RSAFS'].pct_change() * 100 * 12

# Drop rows where any row has NaN
df_RSAFS_after = df_RSAFS_after.dropna()

print(df_RSAFS_after.isnull().sum())

display_side_by_side(
    df_RSAFS_before.head(), df_RSAFS_after.head(), df_RSAFS_before.tail(), df_RSAFS_after.tail(), 
    titles=['Raw Data (Head)', 'Cleaned Data (Head)', 'Raw Data (Tail)', 'Cleaned Data (Tail)']
)

RSAFS    0
dtype: int64


,RSAFS
1992-01-01,159177.0
1992-02-01,159189.0
1992-03-01,158647.0
1992-04-01,159921.0
1992-05-01,160471.0
,RSAFS
1992-02-01,0.090465
1992-03-01,-4.085709
1992-04-01,9.636489
1992-05-01,4.127038


### Supply
- Crude Oil Prices

In [12]:
df_MCOILWTICO_before = pd.DataFrame(fetch_series('MCOILWTICO'))

# Copy the dataframe
df_MCOILWTICO_after = df_MCOILWTICO_before.copy()

# Compute YoY growth
df_MCOILWTICO_after['MCOILWTICO'] = df_MCOILWTICO_after['MCOILWTICO'].pct_change(periods=12) * 100

# Drop rows where any row has NaN
df_MCOILWTICO_after = df_MCOILWTICO_after.dropna()

print(df_MCOILWTICO_after.isnull().sum())

display_side_by_side(
    df_MCOILWTICO_before.head(), df_MCOILWTICO_after.head(), df_MCOILWTICO_before.tail(), df_MCOILWTICO_after.tail(), 
    titles=['Raw Data (Head)', 'Cleaned Data (Head)', 'Raw Data (Tail)', 'Cleaned Data (Tail)']
)

MCOILWTICO    0
dtype: int64


,MCOILWTICO
1986-01-01,22.93
1986-02-01,15.46
1986-03-01,12.61
1986-04-01,12.84
1986-05-01,15.38
,MCOILWTICO
1987-01-01,-18.665504
1987-02-01,14.812419
1987-03-01,45.122918
1987-04-01,45.482866


- Import Price Index

In [13]:
df_IR_before = pd.DataFrame(fetch_series('IR'))

# Copy the dataframe
df_IR_after = df_IR_before['1988-12-01':].copy()

# Compute YoY growth
df_IR_after['IR'] = df_IR_after['IR'].pct_change(periods=12) * 100

# Drop rows where any row has NaN
df_IR_after = df_IR_after.dropna()

print(df_IR_after.isnull().sum())

display_side_by_side(
    df_IR_before.head(), df_IR_after.head(), df_IR_before.tail(), df_IR_after.tail(), 
    titles=['Raw Data (Head)', 'Cleaned Data (Head)', 'Raw Data (Tail)', 'Cleaned Data (Tail)']
)

IR    0
dtype: int64


,IR
1982-09-01,80.0
1982-10-01,NaN
1982-11-01,NaN
1982-12-01,79.9
1983-01-01,NaN
,IR
1989-12-01,2.575588
1990-01-01,1.536773
1990-02-01,2.317881
1990-03-01,1.204819


- Labor Productivity

In [14]:
df_OPHNFB_before = pd.DataFrame(fetch_series('OPHNFB'))

# Copy the dataframe
df_OPHNFB_after = df_OPHNFB_before.copy()

# Compute QoQ annualized growth
df_OPHNFB_after['OPHNFB'] = df_OPHNFB_after['OPHNFB'].pct_change() * 100 * 4

# Convert a quarterly series into a monthly series using spline interpolation
df_OPHNFB_after = df_OPHNFB_after.resample('MS').interpolate(method='spline', order=3)

# Drop rows where any row has NaN
df_OPHNFB_after = df_OPHNFB_after.dropna()

print(df_OPHNFB_after.isnull().sum())

display_side_by_side(
    df_OPHNFB_before.head(), df_OPHNFB_after.head(), df_OPHNFB_before.tail(), df_OPHNFB_after.tail(), 
    titles=['Raw Data (Head)', 'Cleaned Data (Head)', 'Raw Data (Tail)', 'Cleaned Data (Tail)']
)

OPHNFB    0
dtype: int64


,OPHNFB
1947-01-01,22.256
1947-04-01,22.762
1947-07-01,22.065
1947-10-01,22.993
1948-01-01,23.097
,OPHNFB
1947-04-01,9.094177
1947-05-01,-13.205456
1947-06-01,-18.005411
1947-07-01,-12.248484


### Labor Markets
- Average Hourly Earnings

In [15]:
df_CES0500000003_before = pd.DataFrame(fetch_series('CES0500000003'))

# Copy the dataframe
df_CES0500000003_after = df_CES0500000003_before.copy()

# Compute monthly annualized growth
df_CES0500000003_after['CES0500000003'] = df_CES0500000003_after['CES0500000003'].pct_change() * 100 * 12

# Drop rows where any row has NaN
df_CES0500000003_after = df_CES0500000003_after.dropna()

print(df_CES0500000003_after.isnull().sum())

display_side_by_side(
    df_CES0500000003_before.head(), df_CES0500000003_after.head(), df_CES0500000003_before.tail(), df_CES0500000003_after.tail(), 
    titles=['Raw Data (Head)', 'Cleaned Data (Head)', 'Raw Data (Tail)', 'Cleaned Data (Tail)']
)

CES0500000003    0
dtype: int64


,CES0500000003
2006-03-01,20.05
2006-04-01,20.15
2006-05-01,20.13
2006-06-01,20.23
2006-07-01,20.29
,CES0500000003
2006-04-01,5.985037
2006-05-01,-1.191067
2006-06-01,5.961252
2006-07-01,3.559071


- Labor Force Participation Rate

In [16]:
df_CIVPART_before = pd.DataFrame(fetch_series('CIVPART'))

# Copy the dataframe
df_CIVPART_after = df_CIVPART_before.copy()

# # Compute month-over-month difference (not percent change)
# df_CIVPART_after['CIVPART'] = df_CIVPART_after['CIVPART'].diff()

# Drop rows where any row has NaN
df_CIVPART_after = df_CIVPART_after.dropna()

print(df_CIVPART_after.isnull().sum())

display_side_by_side(
    df_CIVPART_before.head(), df_CIVPART_after.head(), df_CIVPART_before.tail(), df_CIVPART_after.tail(), 
    titles=['Raw Data (Head)', 'Cleaned Data (Head)', 'Raw Data (Tail)', 'Cleaned Data (Tail)']
)

CIVPART    0
dtype: int64


,CIVPART
1948-01-01,58.6
1948-02-01,58.9
1948-03-01,58.5
1948-04-01,59.0
1948-05-01,58.3
,CIVPART
1948-01-01,58.6
1948-02-01,58.9
1948-03-01,58.5
1948-04-01,59.0


- Job Openings 

In [17]:
df_JTSJOL_before = pd.DataFrame(fetch_series('JTSJOL'))

# Copy the dataframe
df_JTSJOL_after = df_JTSJOL_before.copy()

# Compute monthly annualized growth
df_JTSJOL_after['JTSJOL'] = df_JTSJOL_after['JTSJOL'].pct_change() * 100 * 12

# Drop rows where any row has NaN
df_JTSJOL_after = df_JTSJOL_after.dropna()

print(df_JTSJOL_after.isnull().sum())

display_side_by_side(
    df_JTSJOL_before.head(), df_JTSJOL_after.head(), df_JTSJOL_before.tail(), df_JTSJOL_after.tail(), 
    titles=['Raw Data (Head)', 'Cleaned Data (Head)', 'Raw Data (Tail)', 'Cleaned Data (Tail)']
)

JTSJOL    0
dtype: int64


,JTSJOL
2000-12-01,5088.0
2001-01-01,5234.0
2001-02-01,5097.0
2001-03-01,4762.0
2001-04-01,4615.0
,JTSJOL
2001-01-01,34.433962
2001-02-01,-31.410011
2001-03-01,-78.869923
2001-04-01,-37.043259


### Monetary Policy
- Federal Funds Effective Rate

In [18]:
df_FEDFUNDS_before = pd.DataFrame(fetch_series('FEDFUNDS'))

# Copy the dataframe
df_FEDFUNDS_after = df_FEDFUNDS_before.copy()

print(df_FEDFUNDS_after.isnull().sum())

display_side_by_side(
    df_FEDFUNDS_before.head(), df_FEDFUNDS_after.head(), df_FEDFUNDS_before.tail(), df_FEDFUNDS_after.tail(), 
    titles=['Raw Data (Head)', 'Cleaned Data (Head)', 'Raw Data (Tail)', 'Cleaned Data (Tail)']
)

FEDFUNDS    0
dtype: int64


,FEDFUNDS
1954-07-01,0.80
1954-08-01,1.22
1954-09-01,1.07
1954-10-01,0.85
1954-11-01,0.83
,FEDFUNDS
1954-07-01,0.80
1954-08-01,1.22
1954-09-01,1.07
1954-10-01,0.85


- Money Suppley (M2)

In [19]:
df_M2SL_before = pd.DataFrame(fetch_series('M2SL'))

# Copy the dataframe
df_M2SL_after = df_M2SL_before.copy()

# Compute monthly annualized growth
df_M2SL_after['M2SL'] = df_M2SL_after['M2SL'].pct_change() * 100 * 12

# Drop rows where any row has NaN
df_M2SL_after = df_M2SL_after.dropna()

print(df_M2SL_after.isnull().sum())

display_side_by_side(
    df_M2SL_before.head(), df_M2SL_after.head(), df_M2SL_before.tail(), df_M2SL_after.tail(), 
    titles=['Raw Data (Head)', 'Cleaned Data (Head)', 'Raw Data (Tail)', 'Cleaned Data (Tail)']
)

M2SL    0
dtype: int64


,M2SL
1959-01-01,286.6
1959-02-01,287.7
1959-03-01,289.2
1959-04-01,290.1
1959-05-01,292.2
,M2SL
1959-02-01,4.605722
1959-03-01,6.256517
1959-04-01,3.734440
1959-05-01,8.686660


- Total Assets of Federal Reserve

In [20]:
df_WALCL_before = pd.DataFrame(fetch_series('WALCL'))

# Copy the dataframe
df_WALCL_after = df_WALCL_before.copy()

# Convert from weekly to monthly (take mean of each month)
df_WALCL_after = df_WALCL_after.resample('MS').mean()

# Create YoY series (% change from a year ago)
df_WALCL_after['WALCL'] = df_WALCL_after['WALCL'].pct_change(periods=12) * 100

# Drop rows where any row has NaN
df_WALCL_after = df_WALCL_after.dropna()

print(df_WALCL_after.isnull().sum())

display_side_by_side(
    df_WALCL_before.head(), df_WALCL_after.head(), df_WALCL_before.tail(), df_WALCL_after.tail(), 
    titles=['Raw Data (Head)', 'Cleaned Data (Head)', 'Raw Data (Tail)', 'Cleaned Data (Tail)']
)

WALCL    0
dtype: int64


,WALCL
2002-12-18,719542.0
2002-12-25,732059.0
2003-01-01,730994.0
2003-01-08,723762.0
2003-01-15,720074.0
,WALCL
2003-12-01,4.685737
2004-01-01,4.859798
2004-02-01,4.480394
2004-03-01,4.406370


### Inflation Expectations
- 5-Year Breakeven Inflation Rate

In [21]:
df_T5YIE_before = pd.DataFrame(fetch_series('T5YIE'))

# Copy the dataframe
df_T5YIE_after = df_T5YIE_before.copy()

# Convert from daily to monthly (take mean of each month)
df_T5YIE_after = df_T5YIE_after.resample('MS').mean()

# Drop rows where any row has NaN
df_T5YIE_after = df_T5YIE_after.dropna()

print(df_T5YIE_after.isnull().sum())

display_side_by_side(
    df_T5YIE_before.head(), df_T5YIE_after.head(), df_T5YIE_before.tail(), df_T5YIE_after.tail(), 
    titles=['Raw Data (Head)', 'Cleaned Data (Head)', 'Raw Data (Tail)', 'Cleaned Data (Tail)']
)

T5YIE    0
dtype: int64


,T5YIE
2003-01-02,1.30
2003-01-03,1.28
2003-01-06,1.31
2003-01-07,1.28
2003-01-08,1.33
,T5YIE
2003-01-01,1.398095
2003-02-01,1.654211
2003-03-01,1.695238
2003-04-01,1.568571


- 10-Year Breakeven Inflation Rate

In [22]:
df_T10YIE_before = pd.DataFrame(fetch_series('T10YIE'))

# Copy the dataframe
df_T10YIE_after = df_T10YIE_before.copy()

# Convert from daily to monthly (take mean of each month)
df_T10YIE_after = df_T10YIE_after.resample('MS').mean()

# Drop rows where any row has NaN
df_T10YIE_after = df_T10YIE_after.dropna()

print(df_T10YIE_after.isnull().sum())

display_side_by_side(
    df_T10YIE_before.head(), df_T10YIE_after.head(), df_T10YIE_before.tail(), df_T10YIE_after.tail(), 
    titles=['Raw Data (Head)', 'Cleaned Data (Head)', 'Raw Data (Tail)', 'Cleaned Data (Tail)']
)

T10YIE    0
dtype: int64


,T10YIE
2003-01-02,1.64
2003-01-03,1.62
2003-01-06,1.63
2003-01-07,1.62
2003-01-08,1.71
,T10YIE
2003-01-01,1.754286
2003-02-01,1.912632
2003-03-01,1.862857
2003-04-01,1.774762


- 1-Year Expected Changes in Inflation Rates
- 5-Year Expected Changes in Inflation Rates

In [23]:
# Read CSV file
df = pd.read_csv(r'C:\Users\jiro2\ds-project-portfolio\data\tbmpx1px5.csv')

# a function to extract the specific data from the original data
def extract_expected_changes_in_inflation_rates(df, col, start='1990-04-01'):
    
    # Combine 'Month' and 'YYYY' columns with a placeholder day (e.g., '1')
    df['Date'] = pd.to_datetime(df['Month'] + ' ' + df['YYYY'].astype(str) + ' 1', format='%B %Y %d')
    
    # Set as index and remove index name
    df = df.set_index('Date')
    df.index.name = None

    # Exteact data after April 1990
    df = df.loc[start:]

    # Extarct the target column
    df = df[[col]]

    return df

df_PX1MD = extract_expected_changes_in_inflation_rates(df.copy(), 'PX_MD')
df_PX5MD = extract_expected_changes_in_inflation_rates(df.copy(), 'PX5_MD')

# Drop rows where any row has NaN
df_PX1MD = df_PX1MD.dropna()
df_PX5MD = df_PX5MD.dropna()

print(df_PX1MD.isnull().sum())
print(df_PX5MD.isnull().sum())

display_side_by_side(
    df.head(), df_PX1MD.head(), df_PX5MD.head(), df.tail(), df_PX1MD.tail(), df_PX5MD.tail(), 
    titles=['Raw Data (Head)', '1-Year Cleaned Data (Head)', '5-Year Cleaned Data (Head)',
           'Raw Data (Tail)', '1-Year Cleaned Data (Tail)', '5-Year Cleaned Data (Tail)']
)

PX_MD    0
dtype: int64
PX5_MD    0
dtype: int64


Raw Data (Head) 
 
 
 
 Month 
 YYYY 
 PX_MD 
 PX5_MD 
 
 
 
 
 0 
 January 
 1978 
 5.2 
 NaN 
 
 
 1 
 February 
 1978 
 6.4 
 NaN 
 
 
 2 
 March 
 1978 
 6.3 
 NaN 
 
 
 3 
 April 
 1978 
 6.7 
 NaN 
 
 
 4 
 May 
 1978 
 6.9 
 NaN 
 
 
 1-Year Cleaned Data (Head) 
 
 
 
 PX_MD 
 
 
 
 
 1990-04-01 
 3.6 
 
 
 1990-05-01 
 3.4 
 
 
 1990-06-01 
 3.8 
 
 
 1990-07-01 
 3.4 
 
 
 1990-08-01 
 4.6 
 
 
 5-Year Cleaned Data (Head) 
 
 
 
 PX5_MD 
 
 
 
 
 1990-04-01 
 4.0 
 
 
 1990-05-01 
 4.3 
 
 
 1990-06-01 
 4.6 
 
 
 1990-07-01 
 4.2 
 
 
 1990-08-01 
 4.6 
 
 
 Raw Data (Tail) 
 
 
 
 Month 
 YYYY 
 PX_MD 
 PX5_MD 
 
 
 
 
 567 
 April 
 2025 
 6.5 
 4.4 
 
 
 568 
 May 
 2025 
 6.6 
 4.2 
 
 
 569 
 June 
 2025 
 5.0 
 4.0 
 
 
 570 
 July 
 2025 
 4.5 
 3.4 
 
 
 571 
 August 
 2025 
 4.8 
 3.5 
 
 
 1-Year Cleaned Data (Tail) 
 
 
 
 PX_MD 
 
 
 
 
 2025-04-01 
 6.5 
 
 
 2025-05-01 
 6.6 
 
 
 2025-06-01 
 5.0 
 
 
 2025-07-01 
 4.5 
 
 
 2025-08-01 
 4.8 
 
 
 5-Year Cleaned Data (Tail) 
 
 
 
 PX5_MD 
 
 
 
 
 2025-04-01 
 4.4 
 
 
 2025-05-01 
 4.2 
 
 
 2025-06-01 
 4.0 
 
 
 2025-07-01 
 3.4 
 
 
 2025-08-01 
 3.5

## Merge All Data

In [24]:
# Merge them on the index, keeping only the intersection (common dates)
df_merged = pd.concat([
    # Inflation
    df_PCEPILFE_after, # Core PCE
    # Unemployment
    df_UNRATE_after, # Unemployment Rate
    df_NROU_after, # Noncyclical Rate of Unemployment
    # Business Cycles and Policy Regimes
    df_USREC_after, # Recession Indicators
    df_ZLB_after, # Zero Lower Bound dummy
    df_COVID, # COVID-19 period dummy
    # Demand
    df_GDPC1_after, # Real GDP
    df_GDPPOT_after, # Real Potential GDP
    df_INDPRO_after, # Industrial Production
    df_RSAFS_after, # Retail Sales
    # Supply
    df_MCOILWTICO_after, # Crude Oil Prices
    df_IR_after, # Import Price Index
    df_OPHNFB_after, # Labor Productivity
    # Labor Markets
    df_CES0500000003_after, # Average Hourly Earnings
    df_CIVPART_after, # Labor Force Participation Rate
    df_JTSJOL_after, # Job Openings
    # Monetary Policy
    df_FEDFUNDS_after, # Federal Funds Effective Rate
    df_M2SL_after, # Money Suppley (M2)
    df_WALCL_after, # Total Assets of Federal Reserve
    # Inflation Expectations
    df_T5YIE_after, # 5-Year Breakeven Inflation Rate
    df_T10YIE_after, # 10-Year Breakeven Inflation Rate
    df_PX1MD, # 1-Year Expected Changes in Inflation Rates
    df_PX5MD # 5-Year Expected Changes in Inflation Rates
], axis=1, join='inner')

df_merged.index = df_merged.index.strftime('%Y-%m-%d')
df_merged = df_merged.loc[:'2025-04-01'].copy()

display_side_by_side(
    df_merged.head().T, df_merged.tail().T, 
    titles=['Merged Cleaned Data (Head)', 'Merged Cleaned Data (Tail)']
)

,2006-04-01,2006-05-01,2006-06-01,2006-07-01,2006-08-01
PCEPILFE_YoY,2.362185,2.408566,2.595788,2.545755,2.671352
PCEPILFE_MoM,3.664276,3.008452,3.029509,1.168840,2.449328
UNRATE,4.700000,4.600000,4.600000,4.700000,4.700000
NROU,5.011462,5.034645,5.030882,5.012245,5.023478
USREC,0.000000,0.000000,0.000000,0.000000,0.000000
ZLB_dummy,0.000000,0.000000,0.000000,0.000000,0.000000
COVID_dummy,0.000000,0.000000,0.000000,0.000000,0.000000
GDPC1,1.035011,0.569404,1.078962,0.599824,2.117888
GDPPOT,2.063006,2.463440,2.461109,1.952011,2.456526
INDPRO,3.552753,0.301573,4.041034,-0.783927,5.208583


## Exploratory Data Analysis (EDA)

## Static Visualization

In [25]:
df_merged.describe().T

,count,mean,std,min,25%,50%,75%,max
PCEPILFE_YoY,229.0,2.146638,1.146317,0.625583,1.470914,1.712774,2.403393,5.649107
PCEPILFE_MoM,229.0,2.130348,1.564865,-3.900723,1.224736,1.815973,2.660362,7.307901
UNRATE,229.0,5.841485,2.191507,3.400000,4.100000,5.000000,7.500000,14.800000
NROU,229.0,4.651827,0.236348,4.303702,4.428717,4.634227,4.869133,5.034645
USREC,229.0,0.087336,0.282946,0.000000,0.000000,0.000000,0.000000,1.000000
ZLB_dummy,229.0,0.475983,0.500517,0.000000,0.000000,0.000000,1.000000,1.000000
COVID_dummy,229.0,0.052402,0.223324,0.000000,0.000000,0.000000,0.000000,1.000000
GDPC1,229.0,1.969871,5.050209,-31.635861,1.340949,2.325556,3.221418,31.320900
GDPPOT,229.0,2.127137,0.218383,1.409802,2.011693,2.141259,2.288302,2.463440
INDPRO,229.0,0.375559,15.321632,-158.901707,-3.371104,0.988368,5.584620,79.018449


In [26]:
# Settings
out_dir = "C:/Users/jiro2/ds-project-portfolio/results/1_DatPrep_EDA"
os.makedirs(out_dir, exist_ok=True)
N_COLS = 6
TITLE_FONTSIZE = 9

def _grid_shape(n_items: int, n_cols: int = 6):
    """Compute (rows, cols) for placing n_items subplots."""
    rows = math.ceil(n_items / n_cols)
    return rows, n_cols

def plot_histograms_grid(df: pd.DataFrame, n_cols: int = N_COLS,
                         out_path: str = None, suptitle: str = "Histograms of All Series"):
    """
    Plot histograms for all numeric columns in df in a grid layout.
    """
    # Numeric columns only
    num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
    n = len(num_cols)
    n_rows, n_cols = _grid_shape(n, n_cols)

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(3.6*n_cols, 2.6*n_rows))
    axes = axes.flatten() if n > 1 else [axes]

    for i, col in enumerate(num_cols):
        ax = axes[i]
        df[col].dropna().hist(ax=ax, bins=30, edgecolor="black", alpha=0.7)
        ax.set_title(str(col), fontsize=TITLE_FONTSIZE, loc="left")
        ax.grid(True, alpha=0.3)

    # Hide unused axes
    for j in range(n, len(axes)):
        axes[j].set_visible(False)

    fig.suptitle(suptitle, fontsize=14)
    plt.tight_layout(rect=[0, 0, 1, 0.95])

    if out_path:
        plt.savefig(out_path, dpi=300, bbox_inches="tight")
        plt.close(fig)
        print(f"Saved histogram grid to {out_path}")
    else:
        plt.show()

hist_path = os.path.join(out_dir, "EDA_histograms_grid_all_series.png")
plot_histograms_grid(df_merged, n_cols=N_COLS, out_path=hist_path,
                     suptitle="Histograms of Macroeconomic Indicators")

Saved histogram grid to C:/Users/jiro2/ds-project-portfolio/results/1_DatPrep_EDA\EDA_histograms_grid_all_series.png


In [27]:
# Settings
out_dir = "C:/Users/jiro2/ds-project-portfolio/results/1_DatPrep_EDA"
os.makedirs(out_dir, exist_ok=True)

N_COLS = 6
TITLE_FONTSIZE = 9

# Dictionary of units for each column
units = {
    'PCEPILFE_YoY': '%',
    'PCEPILFE_MoM': '%',
    'UNRATE': '%',
    'NROU': '%',
    'USREC': '',
    'ZLB_dummy': '',
    'COVID_dummy': '',
    'GDPC1': '%',
    'GDPPOT': '%',
    'INDPRO': '%',
    'RSAFS': '%',
    'MCOILWTICO': '%',
    'IR': '%',
    'OPHNFB': '%',
    'CES0500000003': '%',
    'CIVPART': '%',
    'JTSJOL': '%',
    'FEDFUNDS': '%',
    'M2SL': '%',
    'WALCL': '%',
    'T5YIE': '%',
    'T10YIE': '%',
    'PX_MD': '%',
    'PX5_MD': '%'
}

def _grid_shape(n_items: int, n_cols: int = 6):
    """Compute (rows, cols) for placing n_items subplots."""
    rows = math.ceil(n_items / n_cols)
    return rows, n_cols

def _numeric_columns(df: pd.DataFrame):
    """Return names of numeric columns only."""
    return [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]

def plot_violins_by_group(
    df: pd.DataFrame,
    group_col: str,
    units: dict = None,
    n_cols: int = N_COLS,
    out_path: str = None,
    suptitle: str = None
):
    """
    For each numeric column (except the group column and other dummies),
    draw a violin plot split by unique values of `group_col` (e.g., 0/1).
    One figure per group_col.
    """
    # Ensure group column exists
    if group_col not in df.columns:
        raise ValueError(f"{group_col} not in DataFrame")

    # Ensure numeric dtypes and sort index (optional)
    dfi = df.copy()
    if not isinstance(dfi.index, pd.DatetimeIndex):
        # not required for violins, but keep consistent
        try:
            dfi.index = pd.to_datetime(dfi.index)
        except Exception:
            pass

    # Determine target columns: numeric, excluding the grouping column and the three dummies themselves
    numeric_cols = _numeric_columns(dfi)
    exclude = {group_col, "USREC", "ZLB_dummy", "COVID_dummy"}
    target_cols = [c for c in numeric_cols if c not in exclude]

    # Unique, non-null group levels (e.g., [0,1])
    levels = sorted(dfi[group_col].dropna().unique().tolist())
    if len(levels) < 2:
        print(f"Skip {group_col}: not enough distinct levels to compare.")
        return

    n = len(target_cols)
    n_rows, n_cols = _grid_shape(n, n_cols)

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(3.6*n_cols, 2.6*n_rows), sharex=False, sharey=False)
    axes = axes.flatten() if n > 1 else [axes]

    for i, col in enumerate(target_cols):
        ax = axes[i]

        # Collect data by level
        data = []
        labels = []
        for lv in levels:
            vals = dfi.loc[dfi[group_col] == lv, col].dropna().values
            if len(vals) > 0:
                data.append(vals)
                labels.append(f"{group_col}={lv}")

        # If not enough groups have data, mark as unavailable
        if len(data) < 2:
            ax.set_axis_off()
            ax.text(0.5, 0.5, f"{col}\nNot enough data by {group_col}", ha="center", va="center")
            continue

        # Violin plot (matplotlib)
        parts = ax.violinplot(
            dataset=data,
            showmeans=True,
            showmedians=True,
            widths=0.9
        )

        # Light styling
        for pc in parts.get('bodies', []):
            pc.set_alpha(0.6)

        ax.set_title(str(col), fontsize=TITLE_FONTSIZE, loc="left")
        ax.set_xticks(np.arange(1, len(labels)+1))
        ax.set_xticklabels(labels, rotation=0, fontsize=8)

        # y-axis unit label if provided
        ax.set_ylabel(units.get(col, "") if units is not None else "Value")
        ax.grid(True, alpha=0.3)

    # Hide unused axes
    for j in range(n, len(axes)):
        axes[j].set_visible(False)

    if suptitle is None:
        suptitle = f"Violin Plots by {group_col}"
    fig.suptitle(suptitle, fontsize=14)

    # Common x-label
    fig.text(0.5, 0.04, "Groups", ha="center", fontsize=12)

    plt.tight_layout(rect=[0, 0.07, 1, 0.95])

    if out_path:
        plt.savefig(out_path, dpi=300, bbox_inches="tight")
        plt.close(fig)
        print(f"Saved violin grid to {out_path}")
    else:
        plt.show()

# Assumes df_merged and units dict are already defined
for gcol in ["USREC", "ZLB_dummy", "COVID_dummy"]:
    out_path = os.path.join(out_dir, f"EDA_violins_by_{gcol}.png")
    plot_violins_by_group(
        df=df_merged,
        group_col=gcol,
        units=units, 
        n_cols=N_COLS,
        out_path=out_path,
        suptitle=f"Violin Plots of Macroeconomic Indicators grouped by {gcol}"
    )

Saved violin grid to C:/Users/jiro2/ds-project-portfolio/results/1_DatPrep_EDA\EDA_violins_by_USREC.png
Saved violin grid to C:/Users/jiro2/ds-project-portfolio/results/1_DatPrep_EDA\EDA_violins_by_ZLB_dummy.png
Saved violin grid to C:/Users/jiro2/ds-project-portfolio/results/1_DatPrep_EDA\EDA_violins_by_COVID_dummy.png


In [28]:
# Compute correlation matrix
corr = df_merged.corr()

# Create heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(corr,
            annot=True,        # Show correlation values in cells
            fmt='.2f',         # Format numbers with 2 decimal places
            cmap='coolwarm',   # Color map (red=positive, blue=negative)
            center=0,          # Set 0 at the center of colormap
            cbar=True)         # Show color bar

plt.title('Correlation Heatmap', fontsize=16)
plt.tight_layout()

# Save the figure
out_path = 'C:/Users/jiro2/ds-project-portfolio/results/1_DatPrep_EDA/EDA_correlation_heatmap.png'
plt.savefig(out_path, dpi=300, bbox_inches='tight')

plt.close()
print(f"Saved heatmap to {out_path}")

Saved heatmap to C:/Users/jiro2/ds-project-portfolio/results/1_DatPrep_EDA/EDA_correlation_heatmap.png


## Dynamic Visualization

In [29]:
# Grid settings
n_rows, n_cols = 4, 6
fig, axes = plt.subplots(n_rows, n_cols, figsize=(3.6*n_cols, 2.6*n_rows), sharex=True)
axes = axes.flatten()

# Date formatter for x-axis (YYYY-MM)
date_fmt = mdates.DateFormatter('%Y-%m')
locator = mdates.YearLocator(base=2)

# Plot each column
for i, col in enumerate(df_merged.columns):
    ax = axes[i]
    ax.plot(pd.to_datetime(df_merged.index), df_merged[col], lw=1.2)
    ax.set_title(col, fontsize=9, loc='left')
    ax.set_ylabel(units.get(col, ''), fontsize=8)
    ax.grid(True, alpha=0.3)
    ax.xaxis.set_major_formatter(date_fmt)
    ax.xaxis.set_major_locator(locator)

# Hide unused axes
for j in range(len(df_merged.columns), len(axes)):
    axes[j].set_visible(False)

# Rotate x-axis labels for readability
for ax in axes:
    ax.tick_params(axis='x', labelrotation=45)

# Overall title
fig.suptitle('Time Series of Macroeconomic Indicators', fontsize=18)

# Add common x-label
fig.text(0.5, 0.04, 'YYYY-MM', ha='center', fontsize=12)

# Adjust layout
plt.tight_layout(rect=[0, 0.05, 1, 0.96])

# Save to the specified path
out_path = 'C:/Users/jiro2/ds-project-portfolio/results/1_DatPrep_EDA/EDA_all_columns_timeseries.png'
fig.savefig(out_path, dpi=300, bbox_inches='tight')

plt.close(fig)
print(f'Saved figure to {out_path}')

Saved figure to C:/Users/jiro2/ds-project-portfolio/results/1_DatPrep_EDA/EDA_all_columns_timeseries.png


In [30]:
# Settings
out_dir = "C:/Users/jiro2/ds-project-portfolio/results/1_DatPrep_EDA"
os.makedirs(out_dir, exist_ok=True)
N_COLS = 6                 # number of subplot columns
TITLE_FONTSIZE = 9
FRAC = 0.12                # smoothing span (proportion of data used in each local fit)
ROBUST_IT = 2              # robust iterations to reduce outlier influence

def _grid_shape(n_items: int, n_cols: int = 6):
    """Compute (rows, cols) for placing n_items subplots."""
    rows = math.ceil(n_items / n_cols)
    return rows, n_cols

def _numeric_columns(df: pd.DataFrame):
    """Return names of numeric columns only."""
    return [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]

def loess_all_series(df: pd.DataFrame, frac: float = FRAC, it: int = ROBUST_IT):
    """
    Apply LOESS to all numeric columns in df and return a DataFrame
    with additional '*_loess' columns aligned to the original index.
    """
    df_out = df.copy()
    num_cols = _numeric_columns(df_out)

    # Ensure datetime index
    if not isinstance(df_out.index, pd.DatetimeIndex):
        df_out.index = pd.to_datetime(df_out.index, errors="coerce")
    df_out = df_out.sort_index()

    for col in num_cols:
        y = df_out[col]
        mask = y.notna()
        x_num = np.arange(mask.sum())           # numeric exog for LOESS
        y_num = y[mask].values

        if mask.sum() < 10:
            # Not enough points to smooth reliably
            df_out[f"{col}_loess"] = np.nan
            continue

        # Run LOESS
        fit = lowess(endog=y_num, exog=x_num, frac=frac, it=it, return_sorted=False)
        # Put back on the original index (NaN where original was NaN)
        smoothed = pd.Series(index=y.index, dtype=float)
        smoothed.loc[mask] = fit
        df_out[f"{col}_loess"] = smoothed

    return df_out

import matplotlib.dates as mdates

def plot_loess_grid(df_with_loess: pd.DataFrame, n_cols: int = N_COLS,
                    out_path: str = None, suptitle: str = "LOESS Smoothing (All Series)", units: dict = None):
    """
    Plot all numeric columns with original series and their '*_loess' overlay as a grid.
    """
    # Determine target pairs (original + loess)
    base_cols = [c for c in df_with_loess.columns
                 if (pd.api.types.is_numeric_dtype(df_with_loess[c])
                     and not c.endswith("_loess")
                     and f"{c}_loess" in df_with_loess.columns)]
    n = len(base_cols)
    n_rows, n_cols = _grid_shape(n, n_cols)

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(3.6*n_cols, 2.6*n_rows), sharex=False)
    axes = axes.flatten() if n > 1 else [axes]

    for i, col in enumerate(base_cols):
        ax = axes[i]
        ax.plot(df_with_loess.index, df_with_loess[col], label="Original", alpha=0.5, linewidth=1.0)
        ax.plot(df_with_loess.index, df_with_loess[f"{col}_loess"], label="LOESS", linewidth=2)
        ax.set_title(str(col), fontsize=TITLE_FONTSIZE, loc="left")
        ax.grid(True, alpha=0.3)

        # format x-axis as YYYY-MM, major ticks every 2 years
        ax.xaxis.set_major_locator(mdates.YearLocator(base=2))
        ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))

        # Y axis label: use unit if available
        if units is not None:
            ax.set_ylabel(units.get(col, ""))
        else:
            ax.set_ylabel("Value")

        # Light legend only on first subplot
        if i == 0:
            ax.legend(loc="upper left", fontsize=8, frameon=False)

    # Hide unused axes
    for j in range(n, len(axes)):
        axes[j].set_visible(False)

    # add common x-axis label ---
    fig.text(0.5, 0.04, "YYYY-MM", ha="center", fontsize=12)

    # Rotate x-axis labels for readability
    for ax in axes:
        ax.tick_params(axis='x', labelrotation=45)
    
    fig.suptitle(suptitle + f" | frac={FRAC}, it={ROBUST_IT}", fontsize=14)
    plt.tight_layout(rect=[0, 0.07, 1, 0.95])

    if out_path:
        plt.savefig(out_path, dpi=300, bbox_inches="tight")
        plt.close(fig)
        print(f"Saved LOESS grid to {out_path}")
    else:
        plt.show()

df_loess = loess_all_series(df_merged, frac=FRAC, it=ROBUST_IT)
grid_path = os.path.join(out_dir, "EDA_loess_grid_all_series.png")
plot_loess_grid(df_loess, n_cols=N_COLS, out_path=grid_path,
                suptitle="LOESS of Macroeconomic Indicators",
               units=units)

Saved LOESS grid to C:/Users/jiro2/ds-project-portfolio/results/1_DatPrep_EDA\EDA_loess_grid_all_series.png


In [31]:
# Settings
out_dir = "C:/Users/jiro2/ds-project-portfolio/results/1_DatPrep_EDA"
os.makedirs(out_dir, exist_ok=True)
N_COLS = 6                 # number of subplot columns
TITLE_FONTSIZE = 9
HP_LAMBDA = 129600         # monthly data (Ravn & Uhlig scaling)

def _grid_shape(n_items: int, n_cols: int = 6):
    """Compute (rows, cols) for placing n_items subplots."""
    rows = math.ceil(n_items / n_cols)
    return rows, n_cols

def _numeric_columns(df: pd.DataFrame):
    """Return names of numeric columns only."""
    return [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]

def hpfilter_all_series(df: pd.DataFrame, lam: float = HP_LAMBDA) -> pd.DataFrame:
    df_out = df.copy()
    if not isinstance(df_out.index, pd.DatetimeIndex):
        df_out.index = pd.to_datetime(df_out.index, errors="coerce")
    df_out = df_out.sort_index()

    num_cols = [c for c in df_out.columns if pd.api.types.is_numeric_dtype(df_out[c])]
    for col in num_cols:
        y = df_out[col]
        mask = y.notna()
        if mask.sum() < 10:
            df_out[f"{col}_trend"] = np.nan
            df_out[f"{col}_cycle"] = np.nan
            continue

        # suppress only the sparse efficiency warning during hpfilter
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", SparseEfficiencyWarning)
            cycle, trend = hpfilter(y[mask].astype(float), lamb=lam)

        trend_full = pd.Series(index=y.index, dtype=float)
        cycle_full = pd.Series(index=y.index, dtype=float)
        trend_full.loc[mask] = trend.values
        cycle_full.loc[mask] = cycle.values

        df_out[f"{col}_trend"] = trend_full
        df_out[f"{col}_cycle"] = cycle_full

    return df_out

def plot_hp_trend_grid(df_hp: pd.DataFrame, units: dict = None, n_cols: int = N_COLS,
                       out_path: str = None, suptitle: str = "HP Trend (All Series)"):
    """
    Grid plot: original series overlaid with HP trend for all numeric columns.
    """
    base_cols = [c for c in df_hp.columns
                 if (pd.api.types.is_numeric_dtype(df_hp[c])
                     and not c.endswith("_trend") and not c.endswith("_cycle")
                     and f"{c}_trend" in df_hp.columns)]
    n = len(base_cols)
    n_rows, n_cols = _grid_shape(n, n_cols)

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(3.6*n_cols, 2.6*n_rows), sharex=False)
    axes = axes.flatten() if n > 1 else [axes]

    for i, col in enumerate(base_cols):
        ax = axes[i]
        ax.plot(df_hp.index, df_hp[col], label="Original", alpha=0.5, linewidth=1.0)
        ax.plot(df_hp.index, df_hp[f"{col}_trend"], label="HP Trend", linewidth=2)

        ax.set_title(str(col), fontsize=TITLE_FONTSIZE, loc="left")
        ax.grid(True, alpha=0.3)

        # x-axis: major ticks every 2 years, show YYYY-MM
        ax.xaxis.set_major_locator(mdates.YearLocator(base=2))
        ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))

        # y-axis label: unit if available
        ax.set_ylabel(units.get(col, "") if units is not None else "Value")

        # legend only once
        if i == 0:
            ax.legend(loc="upper left", fontsize=8, frameon=False)

    # Hide unused axes
    for j in range(n, len(axes)):
        axes[j].set_visible(False)

    # Common x-label and rotated ticks
    fig.text(0.5, 0.04, "YYYY-MM", ha="center", fontsize=12)
    for ax in axes:
        ax.tick_params(axis="x", labelrotation=45)

    fig.suptitle(f"{suptitle} | lambda={HP_LAMBDA}", fontsize=14)
    plt.tight_layout(rect=[0, 0.07, 1, 0.95])

    if out_path:
        plt.savefig(out_path, dpi=300, bbox_inches="tight")
        plt.close(fig)
        print(f"Saved HP trend grid to {out_path}")
    else:
        plt.show()

def plot_hp_cycle_grid(df_hp: pd.DataFrame, n_cols: int = N_COLS,
                       out_path: str = None, suptitle: str = "HP Cycle (All Series)"):
    """
    Grid plot: HP cycle (detrended component) for all numeric columns.
    """
    base_cols = [c for c in df_hp.columns
                 if (pd.api.types.is_numeric_dtype(df_hp[c])
                     and f"{c}_cycle" in df_hp.columns
                     and not c.endswith("_trend") and not c.endswith("_cycle"))]
    n = len(base_cols)
    n_rows, n_cols = _grid_shape(n, n_cols)

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(3.6*n_cols, 2.6*n_rows), sharex=False)
    axes = axes.flatten() if n > 1 else [axes]

    for i, col in enumerate(base_cols):
        ax = axes[i]
        ax.plot(df_hp.index, df_hp[f"{col}_cycle"], label="HP Cycle", linewidth=1.5)

        ax.axhline(0.0, color="k", linewidth=0.8, alpha=0.6)  # zero line
        ax.set_title(str(col), fontsize=TITLE_FONTSIZE, loc="left")
        ax.grid(True, alpha=0.3)

        # x-axis: major ticks every 2 years, show YYYY-MM
        ax.xaxis.set_major_locator(mdates.YearLocator(base=2))
        ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))

        # y-axis label for cycles (unitless deviation from trend)
        ax.set_ylabel("Deviation")

        # legend only once
        if i == 0:
            ax.legend(loc="upper left", fontsize=8, frameon=False)

    # Hide unused axes
    for j in range(n, len(axes)):
        axes[j].set_visible(False)

    # Common x-label and rotated ticks
    fig.text(0.5, 0.04, "YYYY-MM", ha="center", fontsize=12)
    for ax in axes:
        ax.tick_params(axis="x", labelrotation=45)

    fig.suptitle(f"{suptitle} | lambda={HP_LAMBDA}", fontsize=14)
    plt.tight_layout(rect=[0, 0.07, 1, 0.95])

    if out_path:
        plt.savefig(out_path, dpi=300, bbox_inches="tight")
        plt.close(fig)
        print(f"Saved HP cycle grid to {out_path}")
    else:
        plt.show()

df_hp = hpfilter_all_series(df_merged, lam=HP_LAMBDA)

trend_path = os.path.join(out_dir, "EDA_hp_trend_grid_all_series.png")
cycle_path = os.path.join(out_dir, "EDA_hp_cycle_grid_all_series.png")

# If you have a `units` dict from earlier, pass it here
plot_hp_trend_grid(df_hp, units=units, n_cols=N_COLS, out_path=trend_path,
                   suptitle="HP Trend of Macroeconomic Indicators")
plot_hp_cycle_grid(df_hp, n_cols=N_COLS, out_path=cycle_path,
                   suptitle="HP Cycle of Macroeconomic Indicators")

Saved HP trend grid to C:/Users/jiro2/ds-project-portfolio/results/1_DatPrep_EDA\EDA_hp_trend_grid_all_series.png
Saved HP cycle grid to C:/Users/jiro2/ds-project-portfolio/results/1_DatPrep_EDA\EDA_hp_cycle_grid_all_series.png


In [32]:
# Settings
out_dir = "C:/Users/jiro2/ds-project-portfolio/results/1_DatPrep_EDA/"
os.makedirs(out_dir, exist_ok=True)
LAGS = 36                      # number of lags for monthly data
N_COLS = 6                     # number of subplot columns in the grid
TITLE_FONTSIZE = 9

def _grid_shape(n_items: int, n_cols: int = 6):
    """Compute (rows, cols) to place n_items subplots with given columns."""
    rows = math.ceil(n_items / n_cols)
    return rows, n_cols

def _safe_name(name: str) -> str:
    """Make a column name filename-safe."""
    return str(name).replace("/", "_").replace("\\", "_").replace(" ", "_")

def plot_grid_acf(df: pd.DataFrame, lags: int = LAGS, n_cols: int = N_COLS,
                  out_path: str = None, suptitle: str = "ACF (All Series)"):
    """Create a grid of ACF subplots for all numeric columns in df and save one figure."""
    cols = list(df.columns)
    n = len(cols)
    n_rows, n_cols = _grid_shape(n, n_cols)

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(3.2*n_cols, 2.6*n_rows))
    axes = axes.flatten() if n > 1 else [axes]

    df.index = pd.to_datetime(df.index)

    for i, col in enumerate(cols):
        ax = axes[i]
        y = df[col].dropna()
        # Guard clause to skip series with too few observations
        if len(y) < max(10, lags // 2):
            ax.set_axis_off()
            ax.text(0.5, 0.5, f"{col}\nNot enough data", ha="center", va="center")
            continue
        plot_acf(y, lags=lags, ax=ax, zero=False, title="")
        ax.set_title(str(col), fontsize=TITLE_FONTSIZE, loc="left")
        ax.set_xlabel("Lags")
        ax.grid(True, alpha=0.3)

    # Hide any unused axes
    for j in range(n, len(axes)):
        axes[j].set_visible(False)

    fig.suptitle(suptitle, fontsize=14)
    plt.tight_layout(rect=[0, 0, 1, 0.95])

    if out_path:
        plt.savefig(out_path, dpi=300, bbox_inches="tight")
        plt.close(fig)
        print(f"Saved ACF grid to {out_path}")
    else:
        plt.show()

def plot_grid_pacf(df: pd.DataFrame, lags: int = LAGS, n_cols: int = N_COLS,
                   out_path: str = None, suptitle: str = "PACF (All Series)"):
    """Create a grid of PACF subplots for all numeric columns in df and save one figure."""
    cols = list(df.columns)
    n = len(cols)
    n_rows, n_cols = _grid_shape(n, n_cols)

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(3.2*n_cols, 2.6*n_rows))
    axes = axes.flatten() if n > 1 else [axes]

    df.index = pd.to_datetime(df.index)
    
    for i, col in enumerate(cols):
        ax = axes[i]
        y = df[col].dropna()
        # Guard clause to skip series with too few observations
        if len(y) < max(10, lags // 2):
            ax.set_axis_off()
            ax.text(0.5, 0.5, f"{col}\nNot enough data", ha="center", va="center")
            continue
        plot_pacf(y, lags=lags, ax=ax, zero=False, method="ywmle", title="")
        ax.set_title(str(col), fontsize=TITLE_FONTSIZE, loc="left")
        ax.set_xlabel("Lags")
        ax.grid(True, alpha=0.3)

    # Hide any unused axes
    for j in range(n, len(axes)):
        axes[j].set_visible(False)

    fig.suptitle(suptitle, fontsize=14)
    plt.tight_layout(rect=[0, 0, 1, 0.95])

    if out_path:
        plt.savefig(out_path, dpi=300, bbox_inches="tight")
        plt.close(fig)
        print(f"Saved PACF grid to {out_path}")
    else:
        plt.show()

acf_grid_path  = os.path.join(out_dir, "EDA_acf_grid_all_series.png")
pacf_grid_path = os.path.join(out_dir, "EDA_pacf_grid_all_series.png")

plot_grid_acf(df_merged, lags=LAGS, n_cols=N_COLS, out_path=acf_grid_path,
              suptitle=f"ACF (All Series) - Lags={LAGS}")
plot_grid_pacf(df_merged, lags=LAGS, n_cols=N_COLS, out_path=pacf_grid_path,
               suptitle=f"PACF (All Series) - Lags={LAGS}")

Saved ACF grid to C:/Users/jiro2/ds-project-portfolio/results/1_DatPrep_EDA/EDA_acf_grid_all_series.png
Saved PACF grid to C:/Users/jiro2/ds-project-portfolio/results/1_DatPrep_EDA/EDA_pacf_grid_all_series.png
